+ Future development plan: 
    -   schedule direct into cobra
    

In [1]:
import pygsheets
import pandas as pd
import numpy as np
import datetime as dt
import time
import filepath
import pandas as pd
import glob
import os
import numpy as np
import filepaths
import send_email 
import pygsheets
import infrastructure
import datetime
import math
import datetime
from datetime import timedelta  
import re
import warnings
import statistics
from datetime import date
from calendar import monthrange
from dateutil.relativedelta import relativedelta
import xlsxwriter
import send_email
from colorama import Fore, Style
import matplotlib.pyplot as plt
import seaborn as sns
import filepath 
import datetime as dt
from scipy.stats import percentileofscore

In [2]:
t2=time.time()
sms = pd.read_csv(filepath.output_folder + 'SS_LC_merged_data.csv')
df = infrastructure.transform_sms_df(sms)
df = df[~df['Send Strategy'].isin(['PT','RT','IT','OT','W','MI','Mi'])]
#testMaster = scheduleMaster[scheduleMaster['Send Strategy'].isin(['PT','RT','IT','OT','CT'])]
#miningMaster = scheduleMaster[scheduleMaster['Send Strategy']=='MI']
#scheduleMaster = scheduleMaster[~(scheduleMaster.index.isin(testMaster.index) | scheduleMaster.index.isin(miningMaster.index))] #create index s time series sorted index
df['Datestring']=df['Date'].dt.strftime('%m.%d.%y') #create datestring variable 
df['CTR']=(df['Clicks']/df['Delivered'])
df['eCPM']=(df['Revenue']*1000/df['Delivered'])
df['JPCTR']=(df['Jump Page Clicks']/df['Delivered'])
df['optout rate']=(df['Optout']/df['Delivered'])
df['CTR Normalized']=(df['CTR']-df['CTR'].min())/(df['CTR'].max()-df['CTR'].min())
df['eCPM Normalized']=(df['eCPM']-df['eCPM'].min())/(df['eCPM'].max()-df['eCPM'].min())
df['CTR50'] = df['CTR Normalized'] + df['eCPM Normalized']
df['Creative Type'] = df['Creative Type'].str.split(' ',expand = True)[0]
# Import Lanina
lanina = infrastructure.get_lanina()
# mamba 
mamba = infrastructure.get_mamba()
    # oppo ctr
temp1= df.groupby(['Segments','Opportunity Cost Send Strategy','Date']).agg({'Revenue':'sum','Clicks':'sum','Delivered':'sum'}).reset_index()
temp1[['rolling Revenue','rolling Clicks','rolling Delivered']] = temp1.groupby('Segments').shift(1).rolling(30, min_periods=5)[['Revenue','Clicks','Delivered']].sum().reset_index(drop=True)
temp1['Dataset_Agg_30day_ctr'] = temp1['rolling Clicks'] / temp1['rolling Delivered']
temp1['Dataset_Agg_30day_ecpm'] = temp1['rolling Revenue'] * 1000/ temp1['rolling Delivered']
dataset_agg_ctr =  temp1[['Segments','Date','Opportunity Cost Send Strategy','Dataset_Agg_30day_ctr','Dataset_Agg_30day_ecpm']]
df = df.merge(dataset_agg_ctr, how = 'left')
df['Opportunity Clicks'] = df['Clicks'] - df['Dataset_Agg_30day_ctr'] * df['Delivered']
df['Opportunity Cost'] = df['Revenue'] - df['Dataset_Agg_30day_ecpm'] * df['Delivered'] /1000  # revenue  - last 30 days average revenue in that drop 
df['opportunity CTR'] = df['Opportunity Clicks'] / df['Delivered']
df['opportunity eCPM'] = df['Opportunity Cost'] * 1000 / df['Delivered']
opportunity_eCPM_series =  df[(df['opportunity eCPM']!=np.inf)&(df['opportunity eCPM'].isna() == False)]['opportunity eCPM']
opportunity_ctr_series = df[(df['opportunity CTR']!=np.inf)&(df['opportunity CTR'].isna() == False)]['opportunity CTR']
# change the following code to the function calculate_oppo_ctr
# I need to calculate 
def calculate_oppo_ctr(df):
    df['opportunity CTR'] = df['Opportunity Clicks'] / df['Delivered']
    #df['opportunity CTR Normalized']=(df['opportunity CTR']-df['opportunity CTR'].min())/(df['opportunity CTR'].max()-df['opportunity CTR'].min())
    df['opportunity CTR Normalized']= percentileofscore(opportunity_ctr_series, df['opportunity CTR'], kind='weak') / 100
    df['opportunity eCPM'] = df['Opportunity Cost'] * 1000 / df['Delivered']
    #df['opportunity eCPM Normalized']=(df['opportunity eCPM']-df['opportunity eCPM'].min())/(df['opportunity eCPM'].max()-df['opportunity eCPM'].min())
    df['opportunity eCPM Normalized']=percentileofscore(opportunity_eCPM_series, df['opportunity eCPM'], kind='weak') / 100 
    df['opportunity CTR50'] = (df['opportunity CTR Normalized'] + df['opportunity eCPM Normalized'] ) /2 
    return df

df = calculate_oppo_ctr(df)


/var/folders/81/9wfkcg6j573945pbm52zxh_80000gn/T/ipykernel_53644/1346726388.py:2: DtypeWarning: Columns (1,2,3,4,5,6,8,9,10,11,12,14,16,25,26,27,28,29,30,31,32,33,34,35,36,43,47,50,54) have mixed types. Specify dtype option on import or set low_memory=False.
  sms = pd.read_csv(filepath.output_folder + 'SS_LC_merged_data.csv')
/Users/liliguo/Desktop/github/SMS_Report/Daily_Reporting/infrastructure.py:60: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  schedule2['Date']=pd.to_datetime(schedule2[0],errors='coerce')
/opt/anaconda3/lib/python3.8/site-packages/pygsheets/worksheet.py:1477: UserWarning: At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.
  warnings.warn('At least one column name in the data frame is an empty

# Select by Opportunity Cost CTR50 (50% Oppr CTR Normalized + 50% Oppr eCPM Normalized)

In [86]:

####################### Content Selection with HTML Selection #######################

# ignore all warnings 
import warnings
warnings.filterwarnings('ignore')





def content_select(sc,pubid,hitpath):
    ccid=''

    # according to pubid and sc to find out sc_dppub from df
    publisher = infrastructure.get_publisher()
    dppub = publisher[publisher['PUBID']==int(pubid)]['NEW DP.DS or DP.sV'].values[0]
    dv = publisher[publisher['PUBID']==int(pubid)]['Sub Vertical'].values[0]
    sc_dppub = sc + "_" + dppub
    
    df['Opp Cost eCPM'] = df['Opportunity Cost'] / df['Delivered'] * 1000

    # Create dataframe of all drops of given hitpath in past 90 days
    df_content = df[(df['Date'] > pd.Timestamp.today() - pd.to_timedelta(90, unit='d')) & (df['Hitpath Offer ID']==int(hitpath)) & (df['Shortcode Name']==sc)]
    df_content = df_content[(df['Creative Type'].isin(lanina['Reporting Content ID'].unique().tolist())) | (df['Hitpath Offer ID']==6272)]

    
    df_content['shortcode_DP.SV'] = df_content['shortcode_DP.SV'] == sc_dppub 
    df_content['data_vertical'] = df_content['Data Vertical'] == dv
    df_content_by_drop = df_content.copy()
    ## get the opportunity CTR50 for each content piece in the same data vertical 
    df_content_other_affliate = df_content[(df_content['shortcode_DP.SV']==False) & (df_content['data_vertical'] == True)]
    df_content_other_affliate = df_content_other_affliate.groupby(['data_vertical','Creative Type','Hitpath Offer ID'], as_index=False).agg({'Delivered': 'sum', 'Opportunity Cost': 'sum','Opportunity Clicks': 'sum','Clicks': 'sum','Revenue':'sum'})
    df_content_other_affliate = calculate_oppo_ctr(df_content_other_affliate)
    df_content_other_affliate = df_content_other_affliate[['Creative Type','opportunity CTR50']]
    df_content_other_affliate = df_content_other_affliate.rename(columns = {'opportunity CTR50':'opportunity CTR50_data_vertical_level'})


    #offer_median_eCPM = df_content['Opp Cost eCPM'].median()
    offer_median_eCPM = df_content['opportunity CTR50'].median()
    df_content_drops = df_content.groupby(['shortcode_DP.SV','Creative Type'], as_index=False).count()
    df_content_drops = df_content_drops[['shortcode_DP.SV','Creative Type', 'Hitpath Offer ID']]
    df_content_drops.columns = ['shortcode_DP.SV','Creative Type','Drops']
    df_content = df_content.groupby(['shortcode_DP.SV','Creative Type','Hitpath Offer ID'], as_index=False).agg({'Delivered': 'sum', 'Opportunity Cost': 'sum','Opportunity Clicks': 'sum','Clicks': 'sum','Revenue':'sum'})
    df_content = calculate_oppo_ctr(df_content)
    df_content['Calculated Opp Cost eCPM'] = df_content['Opportunity Cost'] / df_content['Delivered'] * 1000
    df_content['Calculated CTR'] = df_content['Clicks'] / df_content['Delivered']
    df_content['Calculated opp CTR'] = df_content['Opportunity Clicks'] / df_content['Delivered']
    #df_content['Calculated Opp Cost eCPM'] = df_content['Opportunity Cost'] / df_content['Delivered'] * 1000
    df_content = df_content.merge(df_content_drops, on=['shortcode_DP.SV','Creative Type'])

    # Get the drops for the affiliate account
    df_content_affliate = df_content[df_content['shortcode_DP.SV']==True]
    df_content_affliate_eCPM = df_content_by_drop[df_content_by_drop['shortcode_DP.SV']==True]['opportunity CTR50'].median()
    df_content_drops = df_content_drops[df_content_drops['shortcode_DP.SV']==True]
    df_content_affliate_drops = df_content_affliate.groupby('Creative Type', as_index=False).count()
    #df_content_affliate_eCPM = df_content_affliate['Opportunity Cost'].sum() / df_content_affliate['Delivered'].sum() * 1000
    df_content_affliate = calculate_oppo_ctr(df_content_affliate)
    #df_content_affliate['Calculated Opp Cost eCPM'] = df_content_affliate['Opportunity Cost'] / df_content_affliate['Delivered'] * 1000
    df_content_affliate = df_content_affliate[['shortcode_DP.SV','Creative Type','opportunity CTR50','Calculated Opp Cost eCPM','Calculated CTR','Calculated opp CTR','Drops']]
    df_content_affliate['Meets Threshold'] =  df_content_affliate['opportunity CTR50'] >= df_content_affliate_eCPM

    # Get date last dropped in affiliate account and whether it is in the past 30 days or not
    df_content_dates = df[(df['Date'] > pd.Timestamp.today() - pd.to_timedelta(90, unit='d')) & \
                        (df['Hitpath Offer ID']==int(hitpath)) & \
                        (df['shortcode_DP.SV'] == sc_dppub)]
    df_content_dates = df_content_dates[['Creative Type','Date']]
    df_content_dates = df_content_dates.groupby(['Creative Type'],as_index=False).max()
    cobra_dates = mamba[(mamba['Hitpath Offer ID']==int(hitpath)) & (mamba['shortcode_DP.SV']==sc_dppub) & (mamba['Creative']!='') & (mamba['Date']> df['Date'].max() ) ][['Date','Creative']]
    cobra_dates['Creative'] = cobra_dates['Creative'].str.replace('NEW','')
    cobra_dates['Creative'] = cobra_dates['Creative'].str.replace('*','')
    cobra_dates['Creative'] = cobra_dates['Creative'].str.split('\n')
    cobra_dates = cobra_dates.explode('Creative')
    cobra_dates = cobra_dates.reset_index(drop=True)
    cobra_dates['Creative'] = cobra_dates['Creative'].str.strip()
    cobra_dates = cobra_dates[cobra_dates['Creative']!='']
    #cobra_dates = mamba[(mamba['Hitpath Offer ID']==hitpath) & (mamba['shortcode_DP.SV']==sc_dppub)][['Date','Creative']]
    cobra_dates.columns = ['Date','Creative Type']
    df_content_dates = pd.concat([df_content_dates,cobra_dates],ignore_index=True).drop_duplicates()
    df_content_dates = df_content_dates.groupby(['Creative Type'],as_index=False).max()
    df_content_dates['Recently Dropped'] = df_content_dates['Date'] > pd.Timestamp.today() - pd.to_timedelta(30, unit='d')

    # Get the drops for all other accounts
    df_content_other = df_content[df_content['shortcode_DP.SV']==False]
    df_content_other_eCPM = df_content_by_drop[df_content_by_drop['shortcode_DP.SV']==False]['opportunity CTR50'].median()
    df_content_drops = df_content_drops[df_content_drops['shortcode_DP.SV']==False]
    #df_content_other_eCPM = df_content_other['Opportunity Cost'].sum() / df_content_other['Delivered'].sum() * 1000
    df_content_other = calculate_oppo_ctr(df_content_other)
    df_content_other = df_content_other.merge(df_content_other_affliate, on = 'Creative Type', how = 'left') 
    df_content_other.loc[df_content_other['opportunity CTR50_data_vertical_level'].isna() == False, 'opportunity CTR50'] = df_content_other['opportunity CTR50_data_vertical_level'] * 0.7 +   df_content_other['opportunity CTR50'] * 0.3
    
    #df_content_other_median_eCPM = df_content_other['Revenue CPM (eCPM)'].median()
    df_content_other = df_content_other[['shortcode_DP.SV','Creative Type','opportunity CTR50','Calculated Opp Cost eCPM','Calculated CTR','Calculated opp CTR','Drops']]
    df_content_other['Meets Threshold'] =  df_content_other['opportunity CTR50'] >= df_content_other_eCPM
  
 


    # Merge the above dataframes together 
    df_content = df_content_other.merge(df_content_affliate, how='outer', on='Creative Type').merge(df_content_dates, how='outer', on='Creative Type')
    df_content['Meets Threshold'] = (df_content['Meets Threshold_y'] == True) | ((df_content['Meets Threshold_y'].isnull()) & (df_content['Meets Threshold_x'] == True))
    print('Offer eCPM for Other Accounts: ', round(df_content_other_eCPM,2))
    print('Offer eCPM for Affiliate Account: ', round(df_content_affliate_eCPM,2))
    print('Offer median eCPM: ',round(offer_median_eCPM,2))

    df_content = df_content[['Creative Type', 'opportunity CTR50_x', 'Drops_x',
                                'opportunity CTR50_y', 'Drops_y', 'Date', 'Recently Dropped', 'Meets Threshold']]
    df_content.columns = ['Creative Type','eCPM in Other Accounts','Drops in Other Accounts',
                            'eCPM in Affiliate Account', 'Drops in Affiliate Accounts','Date','Recently Dropped',
                            'Meets Mean Threshold']
    # check if the content has a median eCPM higher than the offer median eCPM
    df_content_by_drop = df_content_by_drop.groupby(['Creative Type'], as_index=False)['opportunity CTR50'].median()
    df_content = df_content.merge(df_content_by_drop, how='left', on='Creative Type')
    df_content = df_content.rename(columns = {'opportunity CTR50':'median eCPM'})
    df_content['Meets Median Threshold'] = df_content['median eCPM'] >= offer_median_eCPM

    # check if the content has an aggregate eCPM in the affiliate account higher than the offer aggregate eCPM in the affiliate account 
    df_content.loc[df_content['eCPM in Affiliate Account'] >= df_content_affliate_eCPM,'Meets Median Threshold'] = True

    # rank is calculated as the average of the aggregate eCPM of the content piece and two times the median eCPM of the content piece 
    df_content['rank'] = (df_content['eCPM in Other Accounts'] + 2 * df_content['median eCPM']) / 3
    # if the content piece has been dropped in the affiliate account in the past 90 days, that aggregate eCPM will be used in the ranking instead of the aggregate for all accounts
    df_content.loc[~df_content['eCPM in Affiliate Account'].isna(),['rank']] = (df_content['eCPM in Affiliate Account'] + 2*df_content['median eCPM'])/3
    df_content = df_content.sort_values('rank', ascending=False)
    df_content = df_content.merge(lanina[['Reporting Content ID','Content Approval Status','Content']], how='left', left_on='Creative Type',right_on = 'Reporting Content ID', copy = False )
    # drop reporting content id column
    df_content = df_content.drop(columns=['Reporting Content ID'])
    # filter out content that is not approved and failed testing 
    df_content = df_content[~(df_content['Content Approval Status'].str.contains('Paused|Not Approved|Failed Testing', na = False)) ]
    df_content_all = df_content.copy()
    if df_content.empty:
        
                    # find Reporting Content ID from la nina based on offer IDs and Type 
        lanina1 =  lanina[~(lanina['Content Approval Status'].str.contains('Paused|Not Approved|Failed Testing', na = False)) ]
        df_content= lanina1[(lanina1['OfferIDs'] == int(hitpath)) & (lanina1['Type'] == sc)][['Reporting Content ID','Content Approval Status', 'Content']]
        if len(df_content) > 0: 
            print('No content available from content selection script, we choose 1 random content from Lanina')
            ccid  = df_content['Reporting Content ID'].values[0]
            df_content_all = df_content.copy()
            print(ccid)
        else:
            
            print('No content available from lanina')
    else:
        # select only content that has been dropped more than 2 times in any account, or has been dropped at least once in the affiliate account it
        df_content['Meets Drop Threshold'] = (df_content['Drops in Other Accounts']>=2) 
        df_content = df_content[((df_content['Meets Drop Threshold']==True) & (df_content['eCPM in Affiliate Account'].isna())) | (~df_content['eCPM in Affiliate Account'].isna())]
        df_content = df_content[(df_content['Meets Mean Threshold']==True) & (df_content['Meets Median Threshold']==True)].sort_values('rank', ascending=False)

    # if no content meets threshold, remove drop number requirement
        if df_content.empty:
            df_content = df_content_all.copy()
            df_content = df_content[(df_content['Meets Mean Threshold']==True) & (df_content['Meets Median Threshold']==True)].sort_values('rank', ascending=False)

        # if after remove number of drop restriction no content meets threshold, reduce mean and median threshold until content is found
        k=0
        while df_content.empty:
            k = k + 1
            print('No offers, decreasing thresholds.')
            df_content = df_content_all.copy()
            df_content_other['Meets Threshold'] =  df_content_other['opportunity CTR50'] >= df_content_other_eCPM - k * 0.1
            df_content_affliate['Meets Threshold'] =  df_content_affliate['opportunity CTR50'] >= df_content_affliate_eCPM - k * 0.1
            df_content['Meets Median Threshold'] = df_content['median eCPM'] >= offer_median_eCPM - k * 0.1
            df_content = df_content[(df_content['Meets Mean Threshold']==True) & (df_content['Meets Median Threshold']==True)].sort_values('rank', ascending=False)

            if k > 20:
                df_content = df_content_all.copy()
                df_content['Meets Drop Threshold'] = (df_content['Drops in Other Accounts'] > 2) 
                df_content = df_content.sort_values('rank', ascending=False)
                break

        df_content = df_content.reset_index(drop=True)
        # If not specifying HTML, remove dates from HTML so gapping is not considered
        
        # Select content, either a random content piece that has not dropped in the past 30 days weighted by the rank or 4 times the rank if opp cost is positive
        # or the one least recently dropped if all have been dropped in the past 30 days
        content = df_content[df_content['Recently Dropped']!=True] # didn't drop in the past 30 days 
        #content.loc[content['rank']>0,'rank'] = content['rank'] * 4 
            
        if content.empty:
            content = df_content[(df_content['Meets Mean Threshold']==True) & (df_content['Meets Median Threshold']==True)]
            selected_creative = content[content['Date'] == content['Date'].min()][['Creative Type']]
        else:
            selected_creative = content[['Creative Type']].sample(n=1,weights=pd.to_numeric(content['rank']) + abs(2*min(pd.to_numeric(content['rank']))))
            #selected_creative = content[['Creative Type']]
        try: 
            ccid = selected_creative['Creative Type'].iloc[0]
        except: 
            ccid = df_content_all['Creative Type'].iloc[0]
        

        
        print('')
        
        print(ccid)
        print('Ranked', df_content.index[df_content['Creative Type'] == ccid][0] + 1, 'out of', len(df_content), 'pieces that meet thresholds')

    return df_content, df_content_all, ccid 


In [133]:
pd.set_option("display.max_rows", None, "display.max_columns", None, 'max_colwidth', None )
sc = 'HZB'
pubid = '461768'
hitpath = '12384'
# according to pubid and sc to find out sc_dppub from df

content_select_result  = content_select(sc,pubid,hitpath)
df_content = content_select_result[0] 
df_content_all = content_select_result[1] 
ccid = content_select_result[2]
df_content


Offer eCPM for Other Accounts:  nan
Offer eCPM for Affiliate Account:  nan
Offer median eCPM:  nan
No content available from content selection script, we choose 1 random content from Lanina
PA.12384.SC.HZB.451971


,Reporting Content ID,Content Approval Status,Content
1088,PA.12384.SC.HZB.451971,Pending - Content Split Testing,HZB: We’re connecting you with our top lenders on {{today_mm_dd_yy}}.\n\nRequest an e-deposit now: {{offer_target_url}}\n\nSTOP to end
1089,PA.12384.SC.HZB.451972,Pending - Content Split Testing,"HZB: {{coalesce|first_name|Hey}}, you’re eligible to request funding before your next paycheck!\n\nStart now: {{offer_target_url}}\n\nSTOP to end"


In [85]:
df_content_all

,Creative Type,eCPM in Other Accounts,Drops in Other Accounts,eCPM in Affiliate Account,Drops in Affiliate Accounts,Date,Recently Dropped,Meets Mean Threshold,median eCPM,Meets Median Threshold,rank,Content Approval Status,Content
0,HHI.5724.SC.HZB-P.1,0.599915,12.0,NaN,NaN,NaT,NaN,True,0.543078,True,0.562024,Approved - Passed,HZB: Pay next month's rent with this: {{offer_target_url}} Click to claim! STOP to end
1,HHI.5724.SC.HZB.451192,0.563435,4.0,NaN,NaN,2023-10-01,True,True,0.559942,True,0.561106,Approved - Passed,HZB: You could pay next month’s rent by answering this quick survey. Start here: {{offer_target_url}} STOP to end
3,HHI.5724.SC.HZB.451871,NaN,NaN,NaN,NaN,2023-10-01,True,False,NaN,False,NaN,Approved - Content Split Testing,HZB: Need help meeting rent? We’ve got just the thing!\n\nView Home Sweepstakes: {{offer_target_url}}\n\nSTOP to end
4,HHI.5724.SC.HZB.451872,NaN,NaN,NaN,NaN,2023-10-01,True,False,NaN,False,NaN,Approved - Content Split Testing,"HZB: Hey {{coalesce|first_name|there}}, need help with your rent or mortgage?\n\nEnter this free sweepstakes: {{offer_target_url}}\n\nSTOP to end"
